In [1]:
import pandas as pd
import sweetviz as sv
import numpy as np
import os

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

In [2]:
data_path = os.path.join('..', 'data', 'SkuChangeRequests.csv')

In [3]:
columns_ru = ['Номер заявки', 'Форма заявки', 'Наименование', 'Единица измерения', 'Изображение', 'Классификация ГОСТ/ТУ', 'Описание', 'Статус', 'Причина отказа', 'Комментарий', 'Ид оферты', 'Наименование оферты', 'Артикул оферты', 'Регион поставки', 'Срок поставки в днях От', 'Срок поставки в днях До', 'Доступное количество От', 'Доступное количество До', 'Исходные характеристики', 'Категория оферты', 'Категория справочника', 'Вид продукции', 'Количество эталонных утвержденных характеристик в категории', 'Количество использованных поставщиком эталонных утвержденных характеристик']
columns_en = ['application_num', 'application_form', 'name', 'measurement_unit', 'image', 'gost_calss', 'description', 'status', 'reason', 'comment' , 'offer_id', 'offer_name', 'offer_number', 'delivery_region', 'delivery_from','delivery_to', 'quantity_from', 'quantity_to', 'init_char', 'offer_category', 'directory_category', 'product_type', 'number_characteristics', 'number_characteristics_supplier']
columns_translator = {ru: eng for ru, eng in zip(columns_ru, columns_en)}

In [4]:
df = pd.read_csv(data_path, sep=',')

In [5]:
df = df.rename(columns=columns_translator)

In [6]:
def sep_reason(reason: str):
    if pd.notna(reason):
        return reason.split(';')
    return reason

In [7]:
reasons_df = df.reason.dropna().apply(sep_reason)

reas = [item for sublist in reasons_df.to_list() for item in sublist]
reas = pd.DataFrame(reas, columns=['reason'])

reject_reasons = {rej: i for i, rej in  enumerate(reas.reason.value_counts().index.to_list())}

reas.reason.value_counts()

предложенных характеристик недостаточно для сопоставления СТЕ                                                                                                                                       30998
наименование заявки должно однозначно описывать предлагаемую СТЕ и содержать значимые отличительные особенности                                                                                     16538
Время редактирования заявки истекло.                                                                                                                                                                15970
необходимо выделить перечень значимых характеристик                                                                                                                                                 11620
изображение должно содержать внешний вид поставляемого товара / товара в упаковке                                                                                                               

In [8]:
reject_reasons

{'предложенных характеристик недостаточно для сопоставления СТЕ': 0,
 'наименование заявки должно однозначно описывать предлагаемую СТЕ и содержать значимые отличительные особенности': 1,
 'Время редактирования заявки истекло.': 2,
 'необходимо выделить перечень значимых характеристик': 3,
 'изображение должно содержать внешний вид поставляемого товара / товара в упаковке': 4,
 'Подробнее о создании оферты и заявки на СТЕ Вы можете ознакомиться в инструкции, которая находится в разделе "Загрузка и создание оферт для классификатора СТЕ"': 5,
 'изображение не должно содержать логотипа, оттисков печатей, водяных знаков, QR-кода, ссылок на веб-ресурсы, а также иных документов': 6,
 'Предложен другой СТЕ': 7,
 'не заполнены поле / поля "Единица измерения"': 8,
 'каждая СТЕ должна загружаться отдельной заявкой': 9,
 'в карточке заявки в поле "Производитель" - должна быть отражена компания производитель, а не страна, в которой был произведен товар': 10,
 'ошибочно указана категория (категория

In [9]:
comments_df = df.comment.dropna().apply(sep_reason)

comm = [item for sublist in comments_df.to_list() for item in sublist]
comm = pd.DataFrame(comm, columns=['comment'])

reject_comm = {com: i for i, com in  enumerate(comm.comment.value_counts().index.to_list())}

comm.comment.value_counts()

Истек регламентный срок ответа                                                                                                                                                                                                                                                 22333
- Количество действующего вещества                                                                                                                                                                                                                                              2822
- Действующее вещество                                                                                                                                                                                                                                                          2793
- Количество в упаковке/объем флакона                                                                                                                                    

In [10]:
reject_comm

{'Истек регламентный срок ответа': 0,
 '- Количество действующего вещества': 1,
 '- Действующее вещество': 2,
 '- Количество в упаковке/объем флакона': 3,
 '- Форма лекарственного средства': 4,
 '- Торговое наименование препарата': 5,
 '"При создании заявки на СТЕ по лекарственным средствам обязательными для заполнения являются следующие характеристики:- Единица измерения действующего вещества': 6,
 '- Срок годности."': 7,
 'Замечания в соответствии с комментарием модератора не устранены. ': 8,
 ' - Количество действующего вещества': 9,
 ' - Действующее вещество': 10,
 ' - Форма лекарственного средства': 11,
 'При создании заявки на СТЕ по лекарственным средствам обязательными для заполнения являются следующие характеристики: - Единица измерения действующего вещества': 12,
 ' - Торговое наименование препарата': 13,
 ' - Количество в упаковке': 14,
 'При создании заявки на СТЕ по лекарственным средствам обязательными для заполнения являются следующие характеристики:- Единица измерения д

In [11]:
len(reject_comm)

18652

In [12]:
REASON_NUM = 55

def get_reject_reason_vector_index(reason):
    if isinstance(reason, list):
        return [reject_reasons.get(reas, None) for reas in reason]
    return []
    
def get_reject_reason_vector(reason_index):
    vec = np.zeros(REASON_NUM, dtype=int)
    vec[reason_index] = 1
    return vec

def get_reject_comment_vector(reason_index):
    vec = np.zeros(REASON_NUM, dtype=int)
    vec[reason_index] = 1
    return vec


In [13]:
df['reason_list'] = df.reason.apply(sep_reason)
df['reason_index'] = df.reason.apply(sep_reason).apply(get_reject_reason_vector_index)
df['reason_vec'] = df.reason.apply(sep_reason).apply(get_reject_reason_vector_index).apply(get_reject_reason_vector)
df['reason_count'] = df.reason.apply(sep_reason).apply(get_reject_reason_vector_index).apply(get_reject_reason_vector).apply(sum)

In [14]:
df[(df.reason.isna())  & (df.comment.isna()) & (df.status == 'Отклонена')]

,application_num,application_form,name,measurement_unit,image,gost_calss,description,status,reason,comment,offer_id,offer_name,offer_number,delivery_region,delivery_from,delivery_to,quantity_from,quantity_to,init_char,offer_category,directory_category,product_type,number_characteristics,number_characteristics_supplier,reason_list,reason_index,reason_vec,reason_count
7384,7384,Полная,Пульсоксиметр,Штука,1.611550e+08,NaN,Система мониторинга пациента Nellcor Bedsi...,Отклонена,NaN,NaN,69544909,Пульсоксиметр,10005941,Все регионы,10,90,1.0,20.0,"[{""name"":""Производитель"",""value"":""Medtronic""},...",Товары-Медицинские товары-Приборы для измерени...,Капнографы-пульсоксиметры,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
7407,7407,Полная,Губка коллагеновая кровоостанавливающая 50х50 ...,Упаковка,1.611617e+08,NaN,NaN,Отклонена,NaN,NaN,69545312,Губка коллагеновая кровоостанавливающая 50х50 ...,7049045,Пермский,1,14,1.0,50.0,"[{""name"":""Производитель"",""value"":""Зеленая Дубр...",Товары-Гемостатики для местного применения (b0...,Гемостатики для местного применения (b02bc),Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
7436,7436,Полная,Аккумулятор,Штука,1.611632e+08,NaN,аккумулятор EDCON,Отклонена,NaN,NaN,69545165,Аккумулятор,1235,Пермский,1,5,1.0,NaN,"[{""name"":""Производитель"",""value"":""edcon""},{""na...",Товары-Товары для автотехники-Запчасти-Аккумул...,Аккумуляторы свинцовые для запуска поршневых д...,Товары,10,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
7554,7554,Полная,Прочие лекарственные средства,Упаковка,1.611761e+08,NaN,1. Слепочная масса силиконовая Стомафлекс Патт...,Отклонена,NaN,NaN,69547134,Прочие лекарственные средства,554566,Москва,1,15,1.0,1.0,"[{""name"":""Производитель"",""value"":""Spofa""},{""na...",Товары-Прочие лекарственные средства-Медицинск...,Прочие лекарственные средства,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
7660,7660,Полная,Пакеты (600 х 750 мм) ПП с индикатором для сбо...,Штука,1.611828e+08,NaN,NaN,Отклонена,NaN,NaN,69547459,Пакеты (600 х 750 мм) ПП с индикатором для сбо...,987545,Москва,1,1,1.0,NaN,"[{""name"":""Производитель"",""value"":""Россия""},{""n...",Товары-Симпатомиметики в комбинации с другими ...,Медицинские мешки,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36964,36964,Полная,Мебель для школы Набор ученич. регулир. 4-6 ро...,Штука,1.789895e+09,NaN,NaN,Отклонена,NaN,NaN,767193607,Мебель для школы Набор ученич. регулир. 4-6 ро...,10-2019,Москва,1,90,1.0,NaN,"[{""name"":""Производитель"",""value"":""no name""},{""...","Товары-Товары для противопожарных, аварийно-сп...",Наборы мебели для образовательных организаций,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
36973,36973,Полная,Электрический накопительный водонагреватель El...,Штука,1.789903e+09,NaN,Электрический накопительный водонагреватель El...,Отклонена,NaN,NaN,767194815,Электрический накопительный водонагреватель E...,27,Москва,5,5,1.0,10.0,"[{""name"":""Производитель"",""value"":""Electrolux""}...",Товары-Товары для ремонта или строительства-Ма...,Электрический водонагреватель,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
37004,37004,Полная,Тент,Штука,1.789951e+09,NaN,тент 2*2,Отклонена,NaN,NaN,767200203,Тент,1605,Москва,10,10,1.0,NaN,"[{""name"":""Производитель"",""value"":""Россия""},{""n...",Товары-Товары для спорта и отдыха-Иные виды сп...,Оборудование,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
37005,37005,Полная,Очки корригирующие,Штука,1.789953e+09,NaN,NaN,Отклонена,NaN,NaN,767200315,"Очки корригирующие Gladiator324, -2",345536-5463,Москва,1,10,1.0,4.0,"[{""name"":""Производитель"",""value"":""Romeo""},{""na...",Товары-Вакцины против ветряной оспы (j07bk)-Ме...,Устройства для коррекции слабовидения,Тов

In [28]:
df[(df.reason.isna()) & (df.status == 'Отклонена')]

,application_num,application_form,name,measurement_unit,image,gost_calss,description,status,reason,comment,offer_id,offer_name,offer_number,delivery_region,delivery_from,delivery_to,quantity_from,quantity_to,init_char,offer_category,directory_category,product_type,number_characteristics,number_characteristics_supplier,reason_list,reason_index,reason_vec,reason_count
4,4,Полная,Cветодиодная вращающаяся голова заливающего св...,Штука,1.585822e+08,NaN,ESTRADA PRO LED MH3610ZW Cветодиодная вращающ...,Отклонена,NaN,Истек регламентный срок ответа,68976691,Cветодиодная вращающаяся голова заливающего ...,110498,Москва,3,30,1.0,20.0,"[{""name"":""Производитель"",""value"":""ESTRADA PRO""...",Товары-Энергетическое оборудование-Осветительн...,Оборудование для создания световых эффектов,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
11,11,Полная,Комплект наградной,Комплект,1.585870e+08,NaN,NaN,Отклонена,NaN,Истек регламентный срок ответа,68977447,Комплект наградной,675гор4,Москва,1,1,1.0,NaN,"[{""name"":""Производитель"",""value"":""ООО \""АРТАНС...",Товары-Награды,Награды,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
26,26,Полная,Лампа светодиод T25 12V 24LED White Маяк,Штука,1.585866e+08,NaN,Лампа светодиод T25 12V 24LED White Маяк. Темп...,Отклонена,NaN,Замечания в соответствии с комментарием модера...,68977383,Лампа светодиод T25 12V 24LED White Маяк,12T25-W/24LED,Пермский,1,1,1.0,NaN,"[{""name"":""Производитель"",""value"":""МАЯК""},{""nam...","Товары-Транспортные средства, расходные матери...",Расходные материалы и запасные части для автом...,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
28,28,Полная,Изолента,Штука,1.585826e+08,NaN,NaN,Отклонена,NaN,Истек регламентный срок ответа,68976422,Изолента,21146627-19,Москва,3,7,1020.0,1020.0,"[{""name"":""Производитель"",""value"":""ИЭК""},{""name...",Товары-Товары для ремонта или строительства-Ма...,Электроизоляционные материалы,Товары,12,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
37,37,Полная,Клапан загрузочный КЗН (У) 1.5,Штука,1.586013e+08,NaN,"Клапан загрузочный навесной КЗН (У) 1,5в компл...",Отклонена,NaN,Замечания в соответствии с комментарием модера...,68978206,Прочее оборудование и инвентарь для санитарног...,Клапан загрузочный КЗН (У) 1.5,Москва,10,20,10.0,300.0,"[{""name"":""Производитель"",""value"":""Россия""},{""n...",Товары-Товары для санитарного и технического с...,Прочее оборудование и инвентарь для санитарног...,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199461,199461,Полная,Акриловая дисперсия Primal SF 16,Упаковка,2.100920e+09,NaN,NaN,Отклонена,NaN,"Данная СТЕ уже есть на портале, осуществите пр...",840536558,Primal,VG-20,Москва,1,10,1.0,2.0,"[{""name"":""Производитель"",""value"":""Waterpaint""}...",Товары-Товары для учреждений культуры и искусс...,Химическая продукция для реставрации,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
199613,199613,Упрощенная,МФУ цветное Konica Minolta bizhub C4050i,NaN,2.100907e+09,NaN,"Скорость системы, А4 \tДо 40/40 стр/мин (моно/...",Отклонена,NaN,"Данная СТЕ уже есть на портале, осуществите пр...",840544043,МФУ цветное Konica Minolta bizhub C4050i,190521-1,Москва,1,1,1.0,250000.0,[],"Товары-Информационно-технологические товары, с...",Многофункциональное устройство (МФУ),Товары,51,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
199674,199674,Полная,Поставка и установка контролера распашных ворот,Штука,2.100928e+09,NaN,NaN,Отклонена,NaN,"Наименование, начинающееся со слов ""Поставка.....",840546141,Поставка и установка контролера распашных ворот,5645212132,Пермский,1,30,1.0,NaN,"[{""name"":""Производитель"",""value"":""DoorHan""},{""...","Товары-Системы охраны, наблюдения, обеспечения...",Ворота,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [18]:
df[df.measurement_unit == 'Рубль'].loc[56747, 'comment']

'Наименование начинающееся со слов "Поставка..." характерно для категории услуг. Измените категорию и описание заявки или уточните наименование оферты для категории "товары".'

In [15]:
df.measurement_unit.value_counts()

Штука                                                                                     154240
Упаковка                                                                                   14336
Комплект                                                                                    6896
Пара (2 шт.)                                                                                4567
Набор                                                                                       3266
Килограмм                                                                                   3194
Метр                                                                                        2488
Литр;^кубический дециметр                                                                   1344
Квадратный метр                                                                             1246
Погонный метр                                                                                971
Рулон                         

In [73]:
df.reason.dropna().apply(sep_reason).apply(len).value_counts()

1     48586
2     18928
3     13172
4      7021
5      3362
6      1519
7       639
8       251
9       114
10       42
11        8
12        4
13        1
Name: reason, dtype: int64

In [40]:
df[df.reason_index == df.comment_index]

,application_num,application_form,name,measurement_unit,image,gost_calss,description,status,reason,comment,offer_id,offer_name,offer_number,delivery_region,delivery_from,delivery_to,quantity_from,quantity_to,init_char,offer_category,directory_category,product_type,number_characteristics,number_characteristics_supplier,reason_list,reason_index,reason_vec,reason_count,comment_list,comment_index,comment_vec,comment_count
0,0,Полная,Изолента,Штука,1.585866e+08,NaN,NaN,Отклонена,наименование заявки должно однозначно описыват...,Истек регламентный срок ответа,68977414,Изолента,21146628-19,Москва,3,7,102.0,102.0,"[{""name"":""Производитель"",""value"":""ИЭК""},{""name...",Товары-Товары для ремонта или строительства-Ма...,Электроизоляционные материалы,Товары,12,0,[наименование заявки должно однозначно описыва...,[1],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,[наименование заявки должно однозначно описыва...,[1],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,1,Полная,Запасные части для компрессора КВД-Г (Комплект),Штука,1.585865e+08,NaN,"1. Кольцо резиновое для муфт артикул: К3, Мате...",Отклонена,каждая СТЕ должна загружаться отдельной заявко...,NaN,68977340,Запасные части для компрессора КВД-Г (Комплект),КВД-Г,Москва,7,10,1.0,2.0,"[{""name"":""Производитель"",""value"":""ООО \""Технок...",Товары-Оборудование-Воздушные компрессоры,Воздушные компрессоры,Товары,0,0,[каждая СТЕ должна загружаться отдельной заявк...,"[9, 15, 1, 5]","[0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",4,[каждая СТЕ должна загружаться отдельной заявк...,"[9, 15, 1, 5]","[0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",4
2,2,Полная,Шпатель медицинский одноразовый,Штука,1.585894e+08,NaN,Шпатель деревянный стерильный одноразовый 100 ...,Отклонена,необходимо указывать единицы измерения в поле ...,NaN,68976829,Медицинские ложки,АГ-66,Пермский,1,10,1.0,150.0,"[{""name"":""Производитель"",""value"":""Россия""},{""n...",Товары-Прямые антикоагулянты (гепарин и его пр...,Прочие расходные материалы медицинского назнач...,Товары,0,0,[необходимо указывать единицы измерения в поле...,"[14, 5]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",2,[необходимо указывать единицы измерения в поле...,"[14, 5]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",2
3,3,Полная,D-димер (уп.: 4 фл. по 3 мл + 4фл. по 9 мл + 2...,Штука,1.585822e+08,NaN,00020008500 D-димер (уп.: 4 фл. по 3 мл + 4фл....,Отклонена,изображение должно содержать внешний вид поста...,NaN,68976787,D-димер (уп.: 4 фл. по 3 мл + 4фл. по 9 мл + 2...,00020008500,Москва,7,45,1.0,NaN,"[{""name"":""Производитель"",""value"":""Instrumentat...",Товары-Таксаны (l01cd)-Медицинские товары-Изде...,Реагенты и расходные материалы к анализаторам,Товары,0,0,[изображение должно содержать внешний вид пост...,[4],"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,[изображение должно содержать внешний вид пост...,[4],"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
4,4,Полная,Cветодиодная вращающаяся голова заливающего св...,Штука,1.585822e+08,NaN,ESTRADA PRO LED MH3610ZW Cветодиодная вращающ...,Отклонена,NaN,Истек регламентный срок ответа,68976691,Cветодиодная вращающаяся голова заливающего ...,110498,Москва,3,30,1.0,20.0,"[{""name"":""Производитель"",""value"":""ESTRADA PRO""...",Товары-Энергетическое оборудование-Осветительн...,Оборудование для создания световых эффектов,Товары,0,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,Полная,Клей для моделей сверхтекучий КЛ-02,Штука,2.101241e+09,NaN,NaN,Принята,NaN,NaN,840569978,Клей,4343-113,Ямало-Ненецкий,20,30,6.0,6.0,"[{""name"":""производитель"",""unitCode"":"""",""newUni...",Товары-Офисные принадлежности-Клеи канцелярские,Клеи канцелярские,Товары,10,0,NaN,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,NaN,[],"[0, 0, 0, 0

In [13]:
my_report = sv.analyze(df)
my_report.show_html('report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
